### Steps for building ANN (Aritificial Neural Network)
- Data Preprocessing (Reading or preparing the data)
- Add input layer
- Random weight initializer
- Add hidden layers
- Select Optimizer, Loss and Performance Metrices (Accuracy, F1_Score, etc)
- Compile the model
- Use model.fit to train the model
- Evaluate the model
- Adjust optimization parameters or model if needed

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential # Importing sequential model
from tensorflow.keras.layers import Flatten, Dense # Importing layers
import pandas as pd

In [2]:
print(tf.__version__)

2.0.0


## Data Preprocessing (Reading or preparing the data)

In [3]:
dataset = pd.read_csv("data/customer_churn_modelling.csv")

In [4]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


#### Dropping unrelated column that doesn't give much information

In [5]:
# X is given variables agains which the prediction will be made
X = dataset.drop(labels=["RowNumber", "CustomerId", "Surname", "Exited"], axis=1) 

# Y is the outcome
Y = dataset["Exited"]

In [6]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [7]:
Y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

#### Since AAN waorks with numerical data. Converting strings to numbers.

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
# Converting Geography

geography_encoder = LabelEncoder()
X["Geography"] = geography_encoder.fit_transform(X["Geography"])

In [10]:
# Now we can see Geography is converted to numbers
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,Female,42,2,0.00,1,1,1,101348.88
1,608,2,Female,41,1,83807.86,1,0,1,112542.58
2,502,0,Female,42,8,159660.80,3,1,0,113931.57
3,699,0,Female,39,1,0.00,2,0,0,93826.63
4,850,2,Female,43,2,125510.82,1,1,1,79084.10


In [11]:
# Now converting Gender

gender_encoder = LabelEncoder()
X["Gender"] = gender_encoder.fit_transform(X["Gender"])

In [12]:
# Now we can see Gender is converted to numbers
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10


In [13]:
# Convert categorical variable into dummy/indicator variables. 

X = pd.get_dummies(X, drop_first=True, columns=["Geography"])
X.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_1,Geography_2
0,619,0,42,2,0.00,1,1,1,101348.88,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,1


#### Feature Standardization

- Feature Standardization is used to bring down the variance in each feature, columns, closer. As our dataset has the column with values varying with each other in a large scale. We will standardize each column so the diffrence in the values in each columns are not too high. Target is to achieve a mean of 0 for each columns.

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [15]:
# Splitting dataset into training and testing set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0, stratify = Y)

In [16]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [17]:
X_train, X_test

(array([[-1.24021723, -1.09665089,  0.77986083, ...,  1.64099027,
         -0.57812007, -0.57504086],
        [ 0.75974873,  0.91186722, -0.27382717, ..., -1.55587522,
          1.72974448, -0.57504086],
        [-1.72725557, -1.09665089, -0.9443559 , ...,  1.1038111 ,
         -0.57812007, -0.57504086],
        ...,
        [-0.51484098,  0.91186722,  0.87565065, ..., -1.01507508,
          1.72974448, -0.57504086],
        [ 0.73902369, -1.09665089, -0.36961699, ..., -1.47887193,
         -0.57812007, -0.57504086],
        [ 0.95663657,  0.91186722, -1.32751517, ...,  0.50945854,
         -0.57812007,  1.73900686]]),
 array([[-1.95320227, -1.09333878,  0.07734873, ..., -0.13121518,
          1.72058015, -0.56888201],
        [ 0.64884591, -1.09333878, -1.42183934, ...,  0.39722862,
          1.72058015, -0.56888201],
        [ 0.13460714,  0.91462959,  2.32613085, ..., -1.46296035,
         -0.58119931,  1.75783377],
        ...,
        [-1.19212889,  0.91462959,  0.17104799, ..., -

# Build ANN

## Add input layer, Random weight initializer, Add hidden layers

In [18]:
X.shape[1]

11

In [19]:
model = Sequential()

# Input Layer. Input all data or all columns in original data. 
# X.shape[1] will give us the count of column in 1st row.
model.add(Dense(X.shape[1], activation="relu", input_dim = X.shape[1])) #X.shape[1] = 11

# Hidden layer. Repeat the test for 128 times.
model.add(Dense(128, activation="relu")) # 128 No of layers. (11 + 1) x 128 = 1536

# Output Layer. Ouput no of layers is expected different outcome -1. Here it is, exited or not exited, 2 - 1 = 1
model.add(Dense(1, activation="sigmoid")) # (128 + 1) x 1 = 129

# Total
# 1536 + 129 = 1797


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 11)                132       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1536      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 1,797
Trainable params: 1,797
Non-trainable params: 0
_________________________________________________________________


In [21]:
X.shape[1]

11

## Select Optimizer, Loss and Performance Metrices (Accuracy, F1_Score, etc) and Compile the model

In [22]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

## Use model.fit to train the model

In [23]:
# batch_size - Represents when to update the weight. For value = 1 it will update the weight for each inputs.
# epoch - Represents how many times it will loop. 
# verbose - Gives to progress bar of how training is going.
model.fit(X_train, Y_train.to_numpy(), batch_size=10, epochs=10, verbose=1)

Train on 8000 samples
Epoch 1/10
8000/8000 [==============================] - 5s 571us/sample - loss: 0.4517 - accuracy: 0.8086
Epoch 2/10
8000/8000 [==============================] - 3s 383us/sample - loss: 0.4095 - accuracy: 0.8275
Epoch 3/10
8000/8000 [==============================] - 3s 398us/sample - loss: 0.3802 - accuracy: 0.8444
Epoch 4/10
8000/8000 [==============================] - 3s 367us/sample - loss: 0.3566 - accuracy: 0.8504
Epoch 5/10
8000/8000 [==============================] - 3s 380us/sample - loss: 0.3469 - accuracy: 0.8569
Epoch 6/10
8000/8000 [==============================] - 3s 349us/sample - loss: 0.3429 - accuracy: 0.8579
Epoch 7/10
8000/8000 [==============================] - 3s 326us/sample - loss: 0.3389 - accuracy: 0.8618 - loss: 0.3389 - accuracy: 
Epoch 8/10
8000/8000 [==============================] - 3s 357us/sample - loss: 0.3375 - accuracy: 0.8612
Epoch 9/10
8000/8000 [==============================] - 3s 324us/sample - loss: 0.3350 - accuracy: 0.8

In [24]:
# Predicting Y value on our test data

Y_pred = model.predict_classes(X_test)

We can see the Y_pred matches the Y_test. So it means our model is predicting correctly. Look at 2nd last value. Its 1 for both Y_pred and Y_test.

In [25]:
Y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [1],
       [0]], dtype=int32)

In [26]:
Y_test

1344    1
8167    0
4747    0
5004    1
3124    1
       ..
9107    0
8249    0
8337    0
6279    1
412     0
Name: Exited, Length: 2000, dtype: int64

## Evaluate the model

In [27]:
model.evaluate(X_test, Y_test.to_numpy())

2000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.3466785798072815, 0.8575]

In [28]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [29]:
confusion_matrix(Y_test, Y_pred)

array([[1520,   73],
       [ 212,  195]])

In [30]:
accuracy_score(Y_test, Y_pred)

0.8575